## Размерность (N, SxD, M)

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

import keras
from keras import layers

2024-12-15 15:18:15.025193: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 15:18:15.025574: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-15 15:18:15.028110: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-15 15:18:15.035174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734250695.047907   22065 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734250695.05

In [2]:
dataset = pd.read_csv("dataset.csv").set_index("id")

In [3]:
dataset

,exam,lecture_1,lecture_2,lecture_3,practice_1,practice_2,practice_3,kt_1,kt_2,is_real
id,,,,,,,,,,
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0
0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0
0,0.4,1.000000,0.714286,0.75,0.500000,0.857143,0.75,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
61,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0
61,0.0,0.666667,0.000000,0.00,0.666667,0.000000,0.00,0.4,0.0,1.0
61,0.0,0.666667,0.000000,0.00,0.500000,0.000000,0.00,0.4,0.0,1.0


In [4]:
semesters_max_count = 8
disciplines_max_count = 8
features_count = 9

In [5]:
data = []
labels = []

In [6]:
zero_discipline_shape = [0.0 for i in range(features_count)]

In [7]:
for id in dataset.index:
    student = [[float(feature) for feature in list(discipline)] for discipline in dataset.loc[id].to_numpy()] 
    
    semesters_count = len(student)//semesters_max_count
    
    if(semesters_count < 2):
        continue
    
    for semester_modif in range(semesters_count-1):
        last_semester = semesters_count-semester_modif
        
        averange_mark = 0
        real_disciplines_count = 0
        for discipline in student[(last_semester-1)*disciplines_max_count: last_semester*disciplines_max_count:]:
            if discipline[-1] == 1.0:
                real_disciplines_count += 1
                mark = round(discipline[0]*5)
                
                averange_mark += mark

        averange_mark = averange_mark / real_disciplines_count
        
        attendanse = False
        if (averange_mark >= 3.0):
            attendanse = True
        
        labels.append(attendanse)
        
        data.append([[feature-0.5 for feature in discipline[:-1:]] for discipline in student[0:(last_semester-1)*disciplines_max_count:]] + 
                    [zero_discipline_shape for i in range(disciplines_max_count*(semesters_max_count-(last_semester-1)))])

In [8]:
train_part = round(len(labels) * 0.85)
test_part = len(labels)-train_part

In [9]:
train_labels = labels[:train_part:]
train_labels = keras.utils.to_categorical(train_labels)

test_labels = labels[train_part::]

train_data = np.array(data[:train_part:])
test_data = np.array(data[train_part::])

In [10]:
model = keras.Sequential()
model.add(layers.GRU(128, input_shape=(disciplines_max_count*semesters_max_count, features_count)))
model.add(layers.Dense(2, activation='softmax'))

2024-12-15 15:18:17.848668: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/atom/Study/ГПО/ASU2304/training/first_model/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=5, batch_size=16)

Epoch 1/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.7145 - loss: 0.4918
Epoch 2/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8279 - loss: 0.3246
Epoch 3/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9395 - loss: 0.1553
Epoch 4/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9599 - loss: 0.1111
Epoch 5/5
887/887 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9843 - loss: 0.0532


In [12]:
test_model = model.predict(test_data)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [17]:
interpreate_test_model = []
for prediction in test_model:
    interpreate_test_model.append(bool(prediction[0] < prediction[1]))

In [18]:
test_compare = []
for i in range(len(test_model)):
    test_compare.append([interpreate_test_model[i], test_labels[i]])

In [19]:
test_compare

[[False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False],
 [False, False

In [23]:
right_predicions = 0

for i in test_compare:
    if i[0] == i[1]:
        right_predicions += 1

accuracy = right_predicions/len(test_labels)

In [24]:
print(accuracy)

0.7379145025968837


In [25]:
model.save("model.keras")